In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

path = "refFlat.txt"
path_fa = "result.fa"
path_name ="name.txt"

In [ ]:
def read_fasta(file_path):
    sequences = {}
    seq_id = None
    current_seq = ""

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):  
                if seq_id is not None:
                    sequences[seq_id] = current_seq
                    current_seq = ""
                seq_id = line[1:]  
            else:
                current_seq += line

        if seq_id is not None and current_seq != "":
            sequences[seq_id] = current_seq

    return sequences



In [ ]:
import re

def split_by_space(text):
    return re.split('\s+', text)

In [ ]:
def gene_data(text_list):
    contents =[]
    for i in range(len(text_list)-1):
        contents.append(text_list[i])
    return contents

In [ ]:
def result(path):
    with open(path, "r", encoding="utf-8") as file:
        lines =file.readlines()
    result = []
    for line in lines:
        line =split_by_space(line)
        contents=gene_data(line)
        result.append(contents)
    return result
    

In [ ]:
def get_name(path):
    name={}
    with open(path, "r", encoding="utf-8") as file:
        lines =file.readlines()
    
    for line in lines:
        line =split_by_space(line)
        key = line[1]
        value = line[0]
        name[key] = value
        
    return name
    

In [ ]:
gene_data =result(path)

In [ ]:
col_names =['geneName','name','chrom','strand','txStart','txEnd','cdsStart','cdsEnd','exonCount','exonStarts','exonEnds']

In [ ]:
gene_df=pd.DataFrame(gene_data,columns=col_names)

In [ ]:
base_data = read_fasta(path_fa)

In [ ]:
gene_name = get_name(path_name)

In [ ]:
gene_name['NM_001357045']

In [ ]:
base_data[gene_name['NM_001357045']]

In [ ]:
gene_df[gene_df['geneName']=='Gpr1']

In [ ]:
gene_df[gene_df['name']=='NR_045921']


In [ ]:
def target_base_CT(seq):
    match = re.findall('\w{17}GG', seq)
    target_list=[]
    for i in range(len(match)):
        target = match[i][0:3]
        if target =='CAA'or target =='CAG'or target =='CGA':
            target_list.append(True)
        
        else:
            target_list.append(False)
    result =[match[s] for s in range(len(match)) if target_list[s]]
    return result

In [ ]:
target_base_CT(base_data[gene_name['NM_001357045']])

In [ ]:
def target_start(target_list,qua):    
    result = []
    for i in target_list:
        matches = re.finditer(i, qua)
        for match in matches:
            start_index = match.start()
            result.append(start_index)
    return result

In [ ]:
target_start(target_base_CT(base_data[gene_name['NM_001357045']]),base_data[gene_name['NM_001357045']])

In [ ]:
def target_gene_start(name,target):
    data = gene_df[gene_df['name']==name]
    num = data['txStart'].astype(int).to_numpy()
    target = np.array(target)
    result = target+num
    #ここ-1が必要かもしれない？
    return result
    

In [ ]:
target_gene_start('NM_001357045',target_start(target_base_CT(base_data[gene_name['NM_001357045']]),base_data[gene_name['NM_001357045']]))

In [ ]:
def in_exon(name,start_exon,):
    data = gene_df[gene_df['name']==name]
    start = data['exonStarts'].to_list()[0]
    end = a= data['exonEnds'].to_list()[0]
    start =np.array(list(start.split(',')))
    end = np.array(list(end.split(',')))
    bool_list  = []
    for i in range(len(start_exon)):
        bool_list.append(False)
        for s in range(len(start)-1):
            if (start[s].astype(int) <= start_exon[i] <=end[s].astype(int)):
                bool_list[-1] = True
    
    result = [start_exon[i] for i in range(len(bool_list)) if bool_list[i]]

    
    return result

In [ ]:
in_exon('NM_001357045',target_gene_start('NM_001357045',target_start(target_base_CT(base_data[gene_name['NM_001357045']]),base_data[gene_name['NM_001357045']])))

In [ ]:
def target_exon():
    input_name = input("遺伝子名: ")
    name_list = gene_df[gene_df['geneName']==input_name]['name'].to_list()
    if 1<len(name_list):
        num_list=[]
        for i in range(len(name_list)):
            name = name_list[i]
            start_num =target_start(target_base_CT(base_data[gene_name[name]]),base_data[gene_name[name]])  
            start =target_gene_start(name,start_num)
            num_list.append(in_exon(name,start))
        result_list = [num for sublist in num_list for num in sublist]
        result = set([x for x in result_list if result_list.count(x) >= (len(name_list))])
    else:
        name = name_list[0]
        start_num =target_start(target_base_CT(base_data[gene_name[name]]),base_data[gene_name[name]])  
        start =target_gene_start(name,start_num)
        result = in_exon(name,start)
    return result

In [ ]:
target_exon()